In [4]:
from typing import Dict
from collections import Counter

RAW = """NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C"""

RULES = Dict[str, str]

class Polymer:
    
    def __init__(self, text)-> None:
        self.polymer, self.rules = self.parse(text)
        
    
    def parse(self, text)->RULES:
        lines = text.split("\n\n")
        polymer, rules = lines[0], lines[-1]
        rules_map = {}
        for rule in rules.splitlines():
            pair, insert = rule.split(" -> ")
            rules_map[pair] = insert
        return polymer, rules_map
    
    def run_step(self):
        
        pairs = [self.polymer[i] + self.polymer[i+1] 
                 for i in range(len(self.polymer)-1)]
        inserts = [self.rules[pair] for pair in pairs]
        
        final_poly = []
        for c, insert in zip(self.polymer, inserts):
            final_poly.append(c)
            final_poly.append(insert)
        final_poly.append(self.polymer[-1])
        self.polymer = "".join(final_poly)
        
    def apply_n_steps(self, num_steps:int)->None:
        
        for i in range(num_steps):
            self.run_step()
    def get_result(self):
        counter = Counter(self.polymer).most_common()
        most, least = counter[0], counter[-1]
        return most[1] - least[1]

P = Polymer(text=RAW)
P.run_step()
assert P.polymer == "NCNBCHB"
P.run_step()
assert P.polymer == "NBCCNBBBCBHCB"
P.run_step()
P.run_step()
assert P.polymer == "NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB"
P.apply_n_steps(6)
assert P.get_result() == 1588

# PART 2
# reviewed JG code/solution, could not crack it

"""
NNCB
(None N) (N N) (N C) (C B) (B None)
(None N) -> (None N)
(N N) -> (N C) (C N)
(N C) -> (N B) (B C)
(C B) -> (C H) (H B)
(B None) -> (B None)
(None N) (N C) (C N) (N B) (B C) (C H) (H B) (B None)
N C N B C H B
NCNBCHB
"""
Rules = Dict[str, str]

class Solution:
    def __init__(self, template:str, rules:Rules)->None:
        self.template = template
        self.rules = {tuple(pair): insert
                      for pair, insert in rules.items()}
        self.counter = Counter()
        self.counter[(None, template[0])] = 1
        self.counter[(template[-1], None)] = 1
        for prev, nxt in zip(template, template[1:]):
            self.counter[(prev, nxt)] = 1
            
    def step(self)->None:
        
        counter = Counter()
        
        for pair in self.counter:
            if None in pair:
                counter[pair] += 1
            else:
                prv, nxt = pair
                insert = self.rules[pair]
                counter[(prv, insert)] += self.counter[pair]
                counter[(insert, nxt)] += self.counter[pair]
        self.counter = counter
        
    def run(self, num_steps:int)->int:
        
        for _ in range(num_steps):
            self.step()
        
        counter = Counter()
        #print("counter is" , self.counter)
        for (prv, nxt), counts in self.counter.items():
            if prv is not None:
                counter[prv] += counts
            if nxt is not None:
                counter[nxt] += counts
        print("new counts", counter)
        mc = counter.most_common()
        double_counts = mc[0][1] - mc[-1][1]
        return double_counts // 2
                
                
    

P = Polymer(RAW)
S = Solution(template=P.polymer, rules=P.rules)
assert S.run(10) == 1588
S = Solution(template=P.polymer, rules=P.rules)
assert S.run(40) == 2188189693529

with open('inputs/day14.txt') as f:
    text = f.read()
    p = Polymer(text=text)
    p.apply_n_steps(num_steps=10)
    print("p1", p.get_result())
    p = Polymer(text=text)
    s = Solution(template=p.polymer, rules=p.rules)
    print('p2', s.run(40))
    

new counts Counter({'B': 3498, 'N': 1730, 'C': 596, 'H': 322})
new counts Counter({'B': 4384079139204, 'N': 2192095604706, 'C': 13195270602, 'H': 7699752146})
p1 2447
new counts Counter({'K': 7267334766344, 'O': 6013123066500, 'F': 5636542979852, 'P': 5395573135438, 'C': 4154479620836, 'H': 4120468460534, 'N': 4098930673364, 'B': 2207642317884, 'V': 1656050543520, 'S': 1231296291218})
p2 3018019237563
